In [1]:
from time import sleep
import pandas as pandas
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait 
import pandas as pd
pd.set_option('display.max_rows', 200)


In [19]:
#we have to puth the path where the webdriver is located, inside the function
driver = webdriver.Chrome('/Users/admin/chromedriver')

In [2]:
def get_df_from_url(url, restaurant_name):
    driver.get(url)
    sleep(5)

    detailed_reviews_list=[]

    restaurant_header = driver.find_elements_by_css_selector('.photoHeader__373c0__YdvQE ')
    restaurant_average_rating = restaurant_header[0].find_element_by_css_selector('.i-stars__373c0__1T6rz').get_attribute('aria-label').split()[0]
    restaurant_reviews_count = restaurant_header[0].find_element_by_css_selector('.text__373c0__2Kxyz').text.split()[0]
    try:
            exp = restaurant_header[0].find_elements_by_css_selector('.text__373c0__2Kxyz.text-bullet--after__373c0__3fS1Z.text-size--large__373c0__3t60B')[-1].text
            restaurant_expensiveness = len(exp)
    except:
            restaurant_expensiveness = 'N/C'

    while True:
        comment_blocs = driver.find_elements_by_css_selector('.review__373c0__13kpL.border-color--default__373c0__3-ifU')   
        
        for i in range(len(comment_blocs)):
            try:
                #we have to change this line as it is not robust
                date = comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz.text-color--mid__373c0__jCeOG.text-align--left__373c0__2XGa-')[3].text.split()[-1]
            except: 
                date = comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz.text-color--mid__373c0__jCeOG.text-align--left__373c0__2XGa-')[2].text.split()[-1]
            rating = comment_blocs[i].find_element_by_css_selector('.i-stars__373c0__1T6rz').get_attribute('aria-label')[0]

            try: 
                username = comment_blocs[i].find_elements_by_css_selector('.user-passport-info .link-size--inherit__373c0__1VFlE')[0].text
            except:
                username = 'Not_Yelp_User'
            
            try:
                photos_for_review = comment_blocs[i].find_elements_by_css_selector('a.link__373c0__1G70M.link-size--inherit__373c0__1VFlE')[1].text[0]
            except:
                photos_for_review = 0
            
            try:
                user_location =  comment_blocs[i].find_elements_by_css_selector('.user-passport-info .text__373c0__2Kxyz')[1].text
            except:
                user_location = 'no_location'

            text_review = comment_blocs[i].find_elements_by_css_selector('.text__373c0__2Kxyz .raw__373c0__3rcx7')[0].text
            user_friends_count = comment_blocs[i].find_elements_by_css_selector('.user-passport-stats__373c0__2LjLz .text__373c0__2Kxyz')[0].text
            user_reviews_count = comment_blocs[i].find_elements_by_css_selector('.user-passport-stats__373c0__2LjLz .text__373c0__2Kxyz')[1].text

            try:
                user_total_image_posted = comment_blocs[i].find_elements_by_css_selector('.user-passport-stats__373c0__2LjLz .text__373c0__2Kxyz')[2].text
                                                                                    
            except:
                user_total_image_posted = 0

            is_real_review = 1
            url = url

            detailed_reviews_list.append([date, username, photos_for_review, rating, text_review, user_location, user_friends_count, user_reviews_count,                                  user_total_image_posted, restaurant_average_rating, restaurant_reviews_count, restaurant_expensiveness, restaurant_name, url, is_real_review])

            
            print(url, i, 'OK', )

        try:
            #navigate to the next page
            driver.find_elements_by_css_selector('.next-link')[0].click()
            sleep(3) 
        except:
            break

    df = pd.DataFrame(detailed_reviews_list,
                                    columns = ['date','username', 'photos_for_review', 'rating',
                                                'text_review', 'user_location', 'user_friends_count',
                                                'user_reviews_count', 'user_total_image_posted', 'restaurant_average_rating',
                                                'restaurant_reviews_count', 'restaurant_expensiveness', 'restaurant_name', 'url', 'is_real_review'])
    return df


In [ ]:
get_df_from_url('https://www.yelp.fr/biz/altrove-paris', 'Altrove' )

In [36]:
#we get the list of links and names previously scraped
dataset = pd.read_csv('Yelp_Paris_list_of_restaurants/restaurants_dataset_only_with_reviews.csv', index_col=0)


In [5]:

df_list = [pd.DataFrame() for i in range(len(dataset))]
error_index_list = []
start = 0

for i in range(start, len(dataset)):
    try:
        df_list[i] = get_df_from_url(dataset['link_to_restaurant'][i], dataset['restaurant_name'][i])
    except:
        error_index_list.append((i, dataset['link_to_restaurant'][i], dataset['restaurant_name'][i]))
        print('restaurant number', i, 'seems to have a problem')
    print('restaurant number ', i, 'OK')

    if i % 50==0 and i != 0 :
        #we create backups each 50 restaurants scraped so that we won't lose data in case of errors in the code
        #also we can interupt and rerun it, we will just have to change the start value at the beginning of the code
        checkpoint = [df_list[k] for k in range(i-50+1, i+1)]
        pd.concat(checkpoint).to_csv('checkpoints/reviews_part'+ str(i) +'.csv')
        pd.DataFrame(error_index_list).to_csv('logfiles/logfile'+ str(i) +'.csv')
        print('checkpoint', i, 'successfully saved')

    if i == len(dataset):
        checkpoint = [df_list[k] for k in range(i-50+1, i+1)]
        pd.concat(checkpoint).to_csv('checkpoints/reviews_part'+ str(i) +'.csv')
        pd.DataFrame(error_index_list).to_csv('logfiles/logfile'+ str(i) +'.csv')


s 1 OK
https://www.yelp.fr/biz/kiccho-paris 2 OK
https://www.yelp.fr/biz/kiccho-paris 3 OK
https://www.yelp.fr/biz/kiccho-paris 4 OK
https://www.yelp.fr/biz/kiccho-paris 5 OK
https://www.yelp.fr/biz/kiccho-paris 6 OK
https://www.yelp.fr/biz/kiccho-paris 7 OK
https://www.yelp.fr/biz/kiccho-paris 8 OK
https://www.yelp.fr/biz/kiccho-paris 9 OK
https://www.yelp.fr/biz/kiccho-paris 10 OK
https://www.yelp.fr/biz/kiccho-paris 11 OK
https://www.yelp.fr/biz/kiccho-paris 12 OK
https://www.yelp.fr/biz/kiccho-paris 13 OK
https://www.yelp.fr/biz/kiccho-paris 14 OK
https://www.yelp.fr/biz/kiccho-paris 15 OK
restaurant number  3649 OK
https://www.yelp.fr/biz/wakoya-paris 0 OK
https://www.yelp.fr/biz/wakoya-paris 1 OK
restaurant number  3650 OK
checkpoint 3650 successfully saved
https://www.yelp.fr/biz/yen-paris 0 OK
https://www.yelp.fr/biz/yen-paris 1 OK
https://www.yelp.fr/biz/yen-paris 2 OK
https://www.yelp.fr/biz/yen-paris 3 OK
https://www.yelp.fr/biz/yen-paris 4 OK
https://www.yelp.fr/biz/yen-par

In [33]:
import glob

path = 'checkpoints/'
# regex that to identify all files with similar_name 
regex = "reviews*"
filenames = glob.glob(path+regex)
# list comprehension to import and concatenate all files
full_dataset = pd.concat((pd.read_csv(file, index_col = 0) for file in filenames))
full_dataset = full_dataset.drop_duplicates()
full_dataset.to_csv('dataset/full_dataset.csv')

In [49]:
full_dataset.head()

,date,username,photos_for_review,rating,text_review,user_location,user_friends_count,user_reviews_count,user_total_image_posted,restaurant_average_rating,restaurant_reviews_count,restaurant_expensiveness,restaurant_name,url,is_real_review
0,21/08/2007,Not_Yelp_User,0,5,Bon retour !\nJe suis revenue dans ce resto ap...,"Levallois-Perret, Hauts-de-Seine",0,4,0,3.0,19,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1
1,140,Not_Yelp_User,0,1,"Très déçue!\n\nDes erreurs dans les commandes,...",Paris,11,250,140,3.0,19,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1
2,26/07/2006,Benjamin D.,0,3,A optimiser...\nCuisine très traditionnelle da...,"Bron, Rhône",0,22,0,3.0,19,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1
3,14/10/2004,Not_Yelp_User,0,3,Brasserie chic\nUne brasserie authentiquement ...,Marseille,11,155,0,3.0,19,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1
4,11/02/2007,Not_Yelp_User,0,4,Tres bien\nPetit diner entre amis. Les plats e...,"Boulogne-Billancourt, Hauts-de-Seine",0,10,0,3.0,19,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1


In [66]:
import glob

path = 'dataset/'
# regex that to identify all files with similar_name 
regex = "dataset*"
filenames = glob.glob(path+regex)
# list comprehension to import and concatenate all files
full_dataset = pd.concat((pd.read_csv(file, index_col = 0) for file in filenames))
full_dataset = full_dataset.drop_duplicates()
full_dataset.to_csv('dataset/full_dataset.csv')

In [68]:
full_dataset.head()

,date,username,photos_for_review,rating,text_review,user_location,user_friends_count,user_reviews_count,user_total_image_posted,restaurant_average_rating,restaurant_reviews_count,restaurant_expensiveness,restaurant_name,url,is_real_review,url_not_recommended
0,21/08/2007,Not_Yelp_User,0,5.0,Bon retour !\nJe suis revenue dans ce resto ap...,"Levallois-Perret, Hauts-de-Seine",0,4,0.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,NaN
1,140,Not_Yelp_User,0,1.0,"Très déçue!\n\nDes erreurs dans les commandes,...",Paris,11,250,140.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,NaN
2,26/07/2006,Benjamin D.,0,3.0,A optimiser...\nCuisine très traditionnelle da...,"Bron, Rhône",0,22,0.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,NaN
3,14/10/2004,Not_Yelp_User,0,3.0,Brasserie chic\nUne brasserie authentiquement ...,Marseille,11,155,0.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,NaN
4,11/02/2007,Not_Yelp_User,0,4.0,Tres bien\nPetit diner entre amis. Les plats e...,"Boulogne-Billancourt, Hauts-de-Seine",0,10,0.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,NaN


In [79]:
fake = pd.read_csv('dataset/dataset_fake.csv', index_col = 0)
real = pd.read_csv('dataset/dataset_real.csv', index_col = 0)


In [223]:
columns_list_ordered = real.columns.values.tolist()
columns_list_ordered.append('url_not_recommended')

In [ ]:
pivot_table = real.loc[:, ['url','restaurant_average_rating',	'restaurant_reviews_count',	'restaurant_expensiveness',	'restaurant_name']].drop_duplicates('url')

In [224]:
fake_merged = (pd.merge(fake, pivot_table, left_on='url', right_on = 'url', how='left'))
fake_dataset_reworked = fake_merged.loc[:, columns_list_ordered]

['date',
 'username',
 'photos_for_review',
 'rating',
 'text_review',
 'user_location',
 'user_friends_count',
 'user_reviews_count',
 'user_total_image_posted',
 'restaurant_average_rating',
 'restaurant_reviews_count',
 'restaurant_expensiveness',
 'restaurant_name',
 'url',
 'is_real_review',
 'url_not_recommended']

In [250]:
full_dataset = pd.concat([real, fake_dataset_reworked])

In [251]:
full_dataset['url_not_recommended'] = full_dataset['url'].apply(lambda url: url.replace("biz/", "not_recommended_reviews/"))



In [255]:
full_dataset.to_csv('full_dataset_reworked.csv')

In [123]:
list_of_missing_urls = (pd.merge(fake, pivot_table, left_on='url', right_on = 'url', how='left'))[mask]['url'].unique()

In [177]:
list_of_missing_url_names = list_of_missing_urls.merge(dataset)

In [179]:
list_of_missing_url_names.to_csv('dataset/missing_restaurants.csv')

2      38254
3      16039
2      10565
1       4043
4       4030
3       3974
14      2494
1       1196
14       758
10       668
4        547
10       147
N/C        8
Name: restaurant_expensiveness, dtype: int64